# Banking Transaction Data - EDA and Preprocessing

This notebook provides exploratory data analysis and preprocessing insights for the PaySim banking transaction dataset.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Configure plotting
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

## 1. Data Loading and Basic Information

In [ ]:
# Load the raw dataset
data_path = Path("../data/raw/PS_20174392719_1491204439457_log.csv")

if data_path.exists():
    df = pd.read_csv(data_path)
    print(f"Dataset loaded successfully!")
    print(f"Shape: {df.shape}")
else:
    print("Dataset not found. Please run the data download script first.")
    print("Run: python data_pipeline/download_data.py")

In [ ]:
# Basic dataset information
if 'df' in locals():
    print("Dataset Info:")
    print(f"Rows: {df.shape[0]:,}")
    print(f"Columns: {df.shape[1]}")
    print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    print("\nColumn Information:")
    print(df.info())

In [ ]:
# Display first few rows
if 'df' in locals():
    print("First 5 rows:")
    display(df.head())
    
    print("\nDataset Description:")
    display(df.describe())

## 2. Data Quality Assessment

In [ ]:
# Check for missing values
if 'df' in locals():
    missing_data = df.isnull().sum()
    missing_percent = (missing_data / len(df)) * 100
    
    missing_df = pd.DataFrame({
        'Missing Count': missing_data,
        'Percentage': missing_percent
    }).sort_values('Missing Count', ascending=False)
    
    print("Missing Values Summary:")
    display(missing_df[missing_df['Missing Count'] > 0])
    
    if missing_df['Missing Count'].sum() == 0:
        print("✓ No missing values found!")

In [ ]:
# Check data types and unique values
if 'df' in locals():
    print("Data Types and Unique Values:")
    for col in df.columns:
        dtype = df[col].dtype
        unique_count = df[col].nunique()
        print(f"{col:15} | {str(dtype):10} | {unique_count:,} unique values")
        
        # Show unique values for categorical columns
        if dtype == 'object' and unique_count < 20:
            print(f"    Values: {df[col].unique()}")
        print()

## 3. Fraud Analysis

In [ ]:
# Fraud distribution analysis
if 'df' in locals() and 'isFraud' in df.columns:
    fraud_counts = df['isFraud'].value_counts()
    fraud_rate = df['isFraud'].mean()
    
    print(f"Fraud Distribution:")
    print(f"Not Fraud: {fraud_counts[0]:,} ({(1-fraud_rate)*100:.2f}%)")
    print(f"Fraud: {fraud_counts[1]:,} ({fraud_rate*100:.2f}%)")
    print(f"\nFraud Rate: {fraud_rate:.4f}")
    
    # Visualize fraud distribution
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Bar chart
    fraud_counts.plot(kind='bar', ax=ax1, color=['skyblue', 'coral'])
    ax1.set_title('Fraud vs. Legitimate Transactions (Count)')
    ax1.set_xlabel('Transaction Type')
    ax1.set_ylabel('Count')
    ax1.set_xticklabels(['Legitimate', 'Fraud'], rotation=0)
    
    # Add count labels on bars
    for i, v in enumerate(fraud_counts.values):
        ax1.text(i, v + max(fraud_counts.values) * 0.01, f'{v:,}', 
                ha='center', va='bottom', fontweight='bold')
    
    # Pie chart
    ax2.pie(fraud_counts.values, labels=['Legitimate', 'Fraud'], 
            colors=['skyblue', 'coral'], autopct='%1.2f%%', startangle=90)
    ax2.set_title('Fraud Distribution (Percentage)')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nClass Imbalance Ratio: {fraud_counts[0]/fraud_counts[1]:.1f}:1")

## 4. Transaction Amount Analysis

In [ ]:
# Amount distribution analysis
if 'df' in locals() and 'amount' in df.columns:
    print("Transaction Amount Statistics:")
    print(df['amount'].describe())
    
    # Create amount distribution plots
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Histogram of amounts
    df['amount'].hist(bins=50, ax=axes[0,0], color='skyblue', alpha=0.7)
    axes[0,0].set_title('Distribution of Transaction Amounts')
    axes[0,0].set_xlabel('Amount')
    axes[0,0].set_ylabel('Frequency')
    axes[0,0].axvline(df['amount'].mean(), color='red', linestyle='--', 
                      label=f'Mean: ${df["amount"].mean():,.2f}')
    axes[0,0].legend()
    
    # Log-scale histogram
    df[df['amount'] > 0]['amount'].apply(np.log10).hist(bins=50, ax=axes[0,1], 
                                                        color='lightgreen', alpha=0.7)
    axes[0,1].set_title('Distribution of Transaction Amounts (Log Scale)')
    axes[0,1].set_xlabel('Log10(Amount)')
    axes[0,1].set_ylabel('Frequency')
    
    # Box plot by fraud status
    if 'isFraud' in df.columns:
        df.boxplot(column='amount', by='isFraud', ax=axes[1,0])
        axes[1,0].set_title('Transaction Amounts by Fraud Status')
        axes[1,0].set_xlabel('Is Fraud')
        axes[1,0].set_ylabel('Amount')
        
        # Comparison of fraud vs legitimate amounts
        fraud_amounts = df[df['isFraud'] == 1]['amount']
        legit_amounts = df[df['isFraud'] == 0]['amount']
        
        axes[1,1].hist([legit_amounts, fraud_amounts], bins=50, alpha=0.7, 
                       label=['Legitimate', 'Fraud'], color=['skyblue', 'coral'])
        axes[1,1].set_title('Amount Distribution: Fraud vs Legitimate')
        axes[1,1].set_xlabel('Amount')
        axes[1,1].set_ylabel('Frequency')
        axes[1,1].legend()
        axes[1,1].set_yscale('log')
    
    plt.tight_layout()
    plt.show()

## 5. Transaction Type Analysis

In [ ]:
# Transaction type analysis
if 'df' in locals() and 'type' in df.columns:
    type_counts = df['type'].value_counts()
    
    print("Transaction Type Distribution:")
    for tx_type, count in type_counts.items():
        percentage = (count / len(df)) * 100
        print(f"{tx_type:12}: {count:,} ({percentage:.2f}%)")
    
    # Visualize transaction types
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Bar chart of transaction types
    type_counts.plot(kind='bar', ax=ax1, color='lightblue')
    ax1.set_title('Transaction Types Distribution')
    ax1.set_xlabel('Transaction Type')
    ax1.set_ylabel('Count')
    ax1.tick_params(axis='x', rotation=45)
    
    # Add count labels
    for i, v in enumerate(type_counts.values):
        ax1.text(i, v + max(type_counts.values) * 0.01, f'{v:,}', 
                ha='center', va='bottom', fontweight='bold')
    
    # Fraud rate by transaction type
    if 'isFraud' in df.columns:
        fraud_by_type = df.groupby('type')['isFraud'].agg(['count', 'sum', 'mean'])
        fraud_by_type.columns = ['Total', 'Fraud_Count', 'Fraud_Rate']
        
        print("\nFraud Rate by Transaction Type:")
        display(fraud_by_type)
        
        fraud_by_type['Fraud_Rate'].plot(kind='bar', ax=ax2, color='coral')
        ax2.set_title('Fraud Rate by Transaction Type')
        ax2.set_xlabel('Transaction Type')
        ax2.set_ylabel('Fraud Rate')
        ax2.tick_params(axis='x', rotation=45)
        
        # Add percentage labels
        for i, v in enumerate(fraud_by_type['Fraud_Rate'].values):
            ax2.text(i, v + max(fraud_by_type['Fraud_Rate'].values) * 0.01, 
                    f'{v:.3f}', ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.show()

## 6. Correlation Analysis

In [ ]:
# Correlation analysis of numerical features
if 'df' in locals():
    # Select numerical columns
    numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    
    if len(numerical_cols) > 1:
        # Calculate correlation matrix
        correlation_matrix = df[numerical_cols].corr()
        
        # Create correlation heatmap
        plt.figure(figsize=(12, 10))
        mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
        
        sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='RdYlBu_r', 
                    center=0, square=True, linewidths=0.5, cbar_kws={"shrink": .5})
        plt.title('Correlation Matrix of Numerical Features')
        plt.tight_layout()
        plt.show()
        
        # Show strongest correlations with fraud (if exists)
        if 'isFraud' in numerical_cols:
            fraud_correlations = correlation_matrix['isFraud'].abs().sort_values(ascending=False)
            fraud_correlations = fraud_correlations[fraud_correlations.index != 'isFraud']
            
            print("\nStrongest correlations with fraud:")
            for feature, corr in fraud_correlations.head(10).items():
                direction = "positive" if correlation_matrix.loc['isFraud', feature] > 0 else "negative"
                print(f"{feature:20}: {corr:.4f} ({direction})")
    else:
        print("Not enough numerical columns for correlation analysis.")

## 7. Balance Analysis

In [ ]:
# Balance analysis (if balance columns exist)
if 'df' in locals():
    balance_cols = [col for col in df.columns if 'balance' in col.lower()]
    
    if balance_cols:
        print(f"Balance columns found: {balance_cols}")
        
        # Calculate balance changes
        if 'oldbalanceOrg' in df.columns and 'newbalanceOrig' in df.columns:
            df['balance_change_orig'] = df['newbalanceOrig'] - df['oldbalanceOrg']
            
        if 'oldbalanceDest' in df.columns and 'newbalanceDest' in df.columns:
            df['balance_change_dest'] = df['newbalanceDest'] - df['oldbalanceDest']
        
        # Plot balance distributions
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        
        if 'oldbalanceOrg' in df.columns:
            df['oldbalanceOrg'].hist(bins=50, ax=axes[0,0], alpha=0.7, color='lightblue')
            axes[0,0].set_title('Original Balance Distribution (Origin)')
            axes[0,0].set_xlabel('Balance')
            axes[0,0].set_ylabel('Frequency')
        
        if 'oldbalanceDest' in df.columns:
            df['oldbalanceDest'].hist(bins=50, ax=axes[0,1], alpha=0.7, color='lightgreen')
            axes[0,1].set_title('Original Balance Distribution (Destination)')
            axes[0,1].set_xlabel('Balance')
            axes[0,1].set_ylabel('Frequency')
        
        if 'balance_change_orig' in df.columns:
            df['balance_change_orig'].hist(bins=50, ax=axes[1,0], alpha=0.7, color='coral')
            axes[1,0].set_title('Balance Change Distribution (Origin)')
            axes[1,0].set_xlabel('Balance Change')
            axes[1,0].set_ylabel('Frequency')
        
        if 'balance_change_dest' in df.columns:
            df['balance_change_dest'].hist(bins=50, ax=axes[1,1], alpha=0.7, color='gold')
            axes[1,1].set_title('Balance Change Distribution (Destination)')
            axes[1,1].set_xlabel('Balance Change')
            axes[1,1].set_ylabel('Frequency')
        
        plt.tight_layout()
        plt.show()
    else:
        print("No balance columns found in the dataset.")

## 8. Time-based Analysis

In [ ]:
# Time-based analysis (if step column exists)
if 'df' in locals() and 'step' in df.columns:
    # Convert step to hours and days
    df['hour'] = df['step'] % 24
    df['day'] = df['step'] // 24
    
    print(f"Time range: Step {df['step'].min()} to {df['step'].max()}")
    print(f"Equivalent to {df['day'].max() + 1} days")
    
    # Plot time-based patterns
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Transactions by hour
    hourly_counts = df['hour'].value_counts().sort_index()
    hourly_counts.plot(kind='bar', ax=axes[0,0], color='skyblue')
    axes[0,0].set_title('Transactions by Hour of Day')
    axes[0,0].set_xlabel('Hour')
    axes[0,0].set_ylabel('Count')
    
    # Transactions by day
    daily_counts = df['day'].value_counts().sort_index()
    daily_counts.plot(ax=axes[0,1], color='lightgreen')
    axes[0,1].set_title('Transactions by Day')
    axes[0,1].set_xlabel('Day')
    axes[0,1].set_ylabel('Count')
    
    # Fraud by hour (if fraud column exists)
    if 'isFraud' in df.columns:
        fraud_by_hour = df.groupby('hour')['isFraud'].mean()
        fraud_by_hour.plot(kind='bar', ax=axes[1,0], color='coral')
        axes[1,0].set_title('Fraud Rate by Hour of Day')
        axes[1,0].set_xlabel('Hour')
        axes[1,0].set_ylabel('Fraud Rate')
        
        # Fraud by day
        fraud_by_day = df.groupby('day')['isFraud'].mean()
        fraud_by_day.plot(ax=axes[1,1], color='orange')
        axes[1,1].set_title('Fraud Rate by Day')
        axes[1,1].set_xlabel('Day')
        axes[1,1].set_ylabel('Fraud Rate')
    
    plt.tight_layout()
    plt.show()
else:
    print("No time column ('step') found for temporal analysis.")

## 9. Customer Analysis

In [ ]:
# Customer-level analysis
if 'df' in locals() and 'nameOrig' in df.columns:
    # Customer transaction statistics
    customer_stats = df.groupby('nameOrig').agg({
        'amount': ['count', 'sum', 'mean', 'std'],
        'isFraud': 'sum' if 'isFraud' in df.columns else 'count'
    }).round(2)
    
    # Flatten column names
    customer_stats.columns = ['_'.join(col).strip() for col in customer_stats.columns]
    customer_stats = customer_stats.reset_index()
    
    print(f"Number of unique customers: {df['nameOrig'].nunique():,}")
    print(f"Average transactions per customer: {df.groupby('nameOrig').size().mean():.2f}")
    
    # Plot customer activity distribution
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Transactions per customer
    transactions_per_customer = df.groupby('nameOrig').size()
    transactions_per_customer.hist(bins=50, ax=ax1, alpha=0.7, color='lightblue')
    ax1.set_title('Distribution of Transactions per Customer')
    ax1.set_xlabel('Number of Transactions')
    ax1.set_ylabel('Number of Customers')
    ax1.axvline(transactions_per_customer.mean(), color='red', linestyle='--', 
               label=f'Mean: {transactions_per_customer.mean():.2f}')
    ax1.legend()
    
    # Total amount per customer
    amount_per_customer = df.groupby('nameOrig')['amount'].sum()
    amount_per_customer.hist(bins=50, ax=ax2, alpha=0.7, color='lightgreen')
    ax2.set_title('Distribution of Total Amount per Customer')
    ax2.set_xlabel('Total Transaction Amount')
    ax2.set_ylabel('Number of Customers')
    ax2.axvline(amount_per_customer.mean(), color='red', linestyle='--', 
               label=f'Mean: ${amount_per_customer.mean():,.2f}')
    ax2.legend()
    
    plt.tight_layout()
    plt.show()
    
    # Top customers by transaction volume
    print("\nTop 10 customers by transaction count:")
    top_customers = transactions_per_customer.sort_values(ascending=False).head(10)
    for customer, count in top_customers.items():
        total_amount = amount_per_customer[customer]
        print(f"{customer}: {count} transactions, ${total_amount:,.2f} total")
else:
    print("No customer identifier column ('nameOrig') found.")

## 10. Preprocessing Summary

In [ ]:
# Load processed data for comparison (if available)
processed_path = Path("../data/processed/transactions_processed.csv")

if processed_path.exists():
    df_processed = pd.read_csv(processed_path)
    
    print("Preprocessing Comparison:")
    print(f"Raw data shape: {df.shape if 'df' in locals() else 'N/A'}")
    print(f"Processed data shape: {df_processed.shape}")
    
    if 'df' in locals():
        print(f"\nColumns added during preprocessing: {set(df_processed.columns) - set(df.columns)}")
        print(f"Columns removed during preprocessing: {set(df.columns) - set(df_processed.columns)}")
    
    # Show sample of processed features
    print("\nSample of processed data:")
    display(df_processed.head())
    
    # Feature correlation in processed data
    if 'isFraud' in df_processed.columns:
        numerical_cols = df_processed.select_dtypes(include=[np.number]).columns
        correlations = df_processed[numerical_cols].corr()['isFraud'].abs().sort_values(ascending=False)
        
        print("\nTop 10 features correlated with fraud (processed data):")
        for feature, corr in correlations[1:11].items():  # Skip isFraud itself
            print(f"{feature:25}: {corr:.4f}")
else:
    print("Processed data not found. Run preprocessing first:")
    print("python data_pipeline/preprocess.py")

## 11. Recommendations for ML Modeling

In [ ]:
# Modeling recommendations based on EDA
print("=" * 60)
print("RECOMMENDATIONS FOR ML MODELING")
print("=" * 60)

if 'df' in locals():
    if 'isFraud' in df.columns:
        fraud_rate = df['isFraud'].mean()
        imbalance_ratio = (1 - fraud_rate) / fraud_rate
        
        print(f"\n1. CLASS IMBALANCE:")
        print(f"   - Fraud rate: {fraud_rate:.4f} ({fraud_rate*100:.2f}%)")
        print(f"   - Imbalance ratio: {imbalance_ratio:.1f}:1")
        
        if imbalance_ratio > 10:
            print(f"   - Recommendation: Use stratified sampling, SMOTE, or class weights")
            print(f"   - Consider ensemble methods like XGBoost with scale_pos_weight")
        
    print(f"\n2. FEATURE ENGINEERING:")
    if 'amount' in df.columns:
        print(f"   - Log-transform amount due to right skewness")
        print(f"   - Create amount bins/categories")
    
    if 'step' in df.columns:
        print(f"   - Extract hour and day features from step")
        print(f"   - Create time-based aggregations")
    
    if 'nameOrig' in df.columns:
        print(f"   - Create customer-level features (transaction count, frequency)")
        print(f"   - Rolling window features for customer behavior")
    
    balance_cols = [col for col in df.columns if 'balance' in col.lower()]
    if len(balance_cols) >= 2:
        print(f"   - Calculate balance changes and ratios")
    
    print(f"\n3. MODEL SELECTION:")
    print(f"   - Primary: XGBoost (handles imbalance, feature interactions)")
    print(f"   - Alternative: Random Forest, LightGBM")
    print(f"   - Consider ensemble of multiple algorithms")
    
    print(f"\n4. EVALUATION METRICS:")
    print(f"   - Primary: AUC-ROC, Precision-Recall AUC")
    print(f"   - Secondary: F1-score, Recall (for fraud detection)")
    print(f"   - Avoid accuracy due to class imbalance")
    
    print(f"\n5. CROSS-VALIDATION:")
    print(f"   - Use stratified k-fold to maintain class distribution")
    print(f"   - Consider time-based splits if temporal patterns matter")
    
    if 'type' in df.columns:
        fraud_by_type = df.groupby('type')['isFraud'].mean()
        high_fraud_types = fraud_by_type[fraud_by_type > fraud_rate * 2].index.tolist()
        if high_fraud_types:
            print(f"\n6. TRANSACTION TYPE INSIGHTS:")
            print(f"   - High-risk types: {high_fraud_types}")
            print(f"   - Consider type-specific models or features")
else:
    print("No data available for recommendations. Please load the dataset first.")

print(f"\n{'='*60}")